## Imports

In [1]:
import pandas as pd
import numpy as np
import re
import os
import geopandas as gpd
import folium
from datetime import datetime, timedelta
import glob
from tqdm.notebook import tqdm
import requests
import time

## Read-in

In [2]:
# Get today's date
today = datetime.now()

today = today - timedelta(days=1)

# Get the date one week ago
one_week_ago = today - timedelta(days=7)

# Function to format the date with the correct ordinal suffix
def format_date_with_ordinal(date):
    day = date.day
    if 4 <= day <= 20 or 24 <= day <= 30:
        suffix = "th"
    else:
        suffix = ["st", "nd", "rd"][day % 10 - 1]
    return date.strftime(f"%b. {day}{suffix}")

# Format both dates
week_ending = format_date_with_ordinal(today)
week_starting = format_date_with_ordinal(one_week_ago)

print(week_starting, week_ending)

Mar. 16th Mar. 23rd


In [3]:
filename = glob.glob('*redfin*.csv')
df_list = []
for file in filename:
    df = pd.read_csv(file)
    df_list.append(df)
    
df = pd.concat(df_list)

In [4]:
df = df.rename(columns={'URL (SEE https://www.redfin.com/buy-a-home/comparative-market-analysis FOR INFO ON PRICING)':'URL'})

In [5]:
remove_string = 'In accordance with local MLS rules, some MLS listings are not included in the download'
df = df[~df['SALE TYPE'].str.contains(remove_string)]

In [6]:
df.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE'],
      dtype='object')

In [7]:
df.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
1,PAST SALE,March-17-2025,Condo/Co-op,711 Lori Dr #213,Palm Springs,FL,33461.0,89325.0,1.0,1.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Palm-Springs/711-Lor...,Beaches MLS,RX-11068798,N,Y,26.645999,-80.092298


In [8]:
print('Input county name:')
county = input()

Input county name:
Palm Beach County


In [9]:
def load_last_week_data_df(filename='last_week_data.csv'):
    try:
        return pd.read_csv(filename)
    except FileNotFoundError:
        return None
    
last_week = load_last_week_data_df()

In [8]:
# new_row = {'SALE TYPE':'PAST SALE','SOLD DATE':'PLACEHOLDER',
#           'PROPERTY TYPE':'Condo/Co-op',
#           'ADDRESS':'PLACEHOLDER',
#           'CITY':'PLACEHOLDER',
#           'STATE OR PROVINCE':'FL',
#           'ZIP OR POSTAL CODE':'PLACEHOLDER',
#           'PRICE':99999999999,
#           'BEDS':9999999999,
#           'BATHS':999999999,
#           'LOCATION':'PLACEHOLDER',
#           'SQUARE FEET':9999999999,
#           'LOT SIZE':None,
#           'YEAR BUILT':'PLACEHOLDER',
#           'DAYS ON MARKET':None,
#           '$/SQUARE FEET':9999999999,
#           'HOA/MONTH':999999999,
#           'STATUS':'Sold',
#           'NEXT OPEN HOUSE START TIME':'PLACEHOLDER',
#           'NEXT OPEN HOUSE END TIME':'PLACEHOLDER',
#           'URL':'Manual Entry',
#           'SOURCE':'Manual Entry',
#           'MLS#':'Manual Entry',
#           'FAVORITE':'N',
#           'INTERESTED':'Y',
#           'LATITUDE':99.999999999,
#           'LONGITUDE':99.999999999}

In [10]:
df['SOLD DATE'] = pd.to_datetime(df['SOLD DATE'])

In [11]:
df2 = df[(df['SOLD DATE'] > one_week_ago) & (df['SOLD DATE'] < today)]

In [12]:
df2['SOLD DATE'].value_counts()

SOLD DATE
2025-03-17    37
2025-03-21    27
2025-03-20    22
2025-03-18    20
2025-03-19    12
Name: count, dtype: int64

In [13]:
df2 = df2.sort_values(by='PRICE',ascending=False)

In [69]:
# df2 = df2.drop(index=64)

In [14]:
df_top_ten = df2.head(10)

In [15]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE
48,PAST SALE,2025-03-21,Condo/Co-op,1000 S Ocean Blvd #701,Boca Raton,FL,33432.0,13000000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Beaches MLS,RX-11045446,N,Y,26.336721,-80.071445
94,PAST SALE,2025-03-20,Condo/Co-op,570 Ocean Dr #902,Juno Beach,FL,33408.0,3050000.0,3.0,4.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Beaches MLS,RX-11020661,N,Y,26.874985,-80.052079
5,PAST SALE,2025-03-20,Condo/Co-op,20155 Boca West Dr Unit C404,Boca Raton,FL,33434.0,2999000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11054302,N,Y,26.377420,-80.164122
12,PAST SALE,2025-03-21,Condo/Co-op,222 Lakeview Ave Unit 2s,West Palm Beach,FL,33401.0,2750000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/222-...,Beaches MLS,RX-11052939,N,Y,26.705667,-80.050960
50,PAST SALE,2025-03-17,Condo/Co-op,701 E Camino Real Unit 8-E,Boca Raton,FL,33432.0,1815000.0,3.0,3.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11042942,N,Y,26.339459,-80.076301
16,PAST SALE,2025-03-21,Condo/Co-op,3720 S Ocean Blvd #507,Highland Beach,FL,33487.0,1560000.0,2.0,2.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11060009,N,Y,26.408254,-80.066146
66,PAST SALE,2025-03-17,Condo/Co-op,801 S Olive Ave #1503,West Palm Beach,FL,33401.0,1550000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-11036395,N,Y,26.705820,-80.052488
79,PAST SALE,2025-03-17,Condo/Co-op,131 Ocean Grande Blvd #402,Jupiter,FL,33477.0,1400000.0,3.0,3.5,...,Sold,NaN,NaN,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,Beaches MLS,RX-11016851,N,Y,26.919651,-80.066425
56,PAST SALE,2025-03-18,Condo/Co-op,400 Seasage Dr #701,Delray Beach,FL,33483.0,1375000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Beaches MLS,RX-11040968,N,Y,26.454271,-80.063637
107,PAST SALE,2025-03-21,Condo/Co-op,120 S Ocean Blvd Unit 1-D,Delray Beach,FL,33483.0,1275000.0,2.0,2.0,...,Sold,NaN,NaN,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Beaches MLS,RX-10988688,N,Y,26.458917,-80.059459


In [12]:
# df_top_ten = df2.head(10)

## Collect Agent Information

In [16]:
def process_response(url):
    try:
        
        base_url = "https://redfin-com-data.p.rapidapi.com/properties/detail-by-url"

        querystring = {"url":url}

        headers = {
            "x-rapidapi-key": "00191da588msh8450293d26e3515p1bbd40jsn56510b513b61",
            "x-rapidapi-host": "redfin-com-data.p.rapidapi.com"
        }

        response = requests.get(base_url, headers=headers, params=querystring)

#         print(response.status_code)
#         print(url)
        time.sleep(0.2)

        response.raise_for_status()
        data = response.json()

        # Initialize default values for listing and buyer agents data
        list_agent_name_1 = None
        list_broker_name_1 = None
        list_agent_name_2 = None
        list_broker_name_2 = None
        
        buy_agent_name_1 = None
        buy_broker_name_1 = None
        buy_agent_name_2 = None
        buy_broker_name_2 = None
        
        # Extract listing agents data if it exists
        listing_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('listingAgents', [])
        if len(listing_agents) > 0:
            list_agent_name_1 = listing_agents[0].get('agentInfo', {}).get('agentName')
            list_broker_name_1 = listing_agents[0].get('brokerName')
        if len(listing_agents) > 1:
            list_agent_name_2 = listing_agents[1].get('agentInfo', {}).get('agentName')
            list_broker_name_2 = listing_agents[1].get('brokerName')
            
        # Extract listing agents data if it exists
        buying_agents = data.get('data', {}).get('mainHouseInfoPanelInfo', {}).get('mainHouseInfo', {}).get('buyingAgents', [])
        if len(buying_agents) > 0:
            buy_agent_name_1 = buying_agents[0].get('agentInfo', {}).get('agentName')
            buy_broker_name_1 = buying_agents[0].get('brokerName')
        if len(buying_agents) > 1:
            buy_agent_name_2 = buying_agents[1].get('agentInfo', {}).get('agentName')
            buy_broker_name_2 = buying_agents[1].get('brokerName')


        return {
            'url': url,
            'list_agent_name_1': list_agent_name_1,
            'list_broker_name_1': list_broker_name_1,
            'list_agent_name_2': list_agent_name_2,
            'list_broker_name_2': list_broker_name_2,
            'buy_agent_name_1': buy_agent_name_1,
            'buy_broker_name_1': buy_broker_name_1,
            'buy_agent_name_2': buy_agent_name_2,
            'buy_broker_name_2': buy_broker_name_2,
        }
    except requests.exceptions.RequestException as e:
        print(f"Error fetching data from {url}: {e}")
        return {
            'url': url,
            'list_agent_name_1': None,
            'list_broker_name_1': None,
            'list_agent_name_2': None,
            'list_broker_name_2': None,
            'buy_agent_name_1': None,
            'buy_broker_name_1': None,
            'buy_agent_name_2': None,
            'buy_broker_name_2': None,
        }

In [17]:
response_list = []

for index, row in df_top_ten.iterrows():
    response_list.append(process_response(row['URL']))

In [18]:
response_df = pd.DataFrame(response_list)

In [19]:
merged_df = pd.merge(left=df_top_ten,left_on='URL',right=response_df, right_on='url')

In [20]:
df_top_ten = merged_df

In [21]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,...,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2
0,PAST SALE,2025-03-21,Condo/Co-op,1000 S Ocean Blvd #701,Boca Raton,FL,33432.0,13000000.0,3.0,3.5,...,-80.071445,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Senada Adzem,Douglas Elliman,None,None,Jonathan Postma,Coldwell Banker/BR,None,None
1,PAST SALE,2025-03-20,Condo/Co-op,570 Ocean Dr #902,Juno Beach,FL,33408.0,3050000.0,3.0,4.5,...,-80.052079,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Carla Christenson,One Sotheby's International Re,Richard Powers,One Sotheby's International Re,Andrea Roth,Echo Fine Properties,None,None
2,PAST SALE,2025-03-20,Condo/Co-op,20155 Boca West Dr Unit C404,Boca Raton,FL,33434.0,2999000.0,2.0,2.5,...,-80.164122,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Nancy Ghen,Coldwell Banker/BR,None,None,Rebecca Spooner,Siemens Group Realty Corp.,None,None
3,PAST SALE,2025-03-21,Condo/Co-op,222 Lakeview Ave Unit 2s,West Palm Beach,FL,33401.0,2750000.0,3.0,3.5,...,-80.050960,https://www.redfin.com/FL/West-Palm-Beach/222-...,Patrick Lafferty,Compass Florida LLC,None,None,Seth Koretzky,Compass Florida LLC,None,None
4,PAST SALE,2025-03-17,Condo/Co-op,701 E Camino Real Unit 8-E,Boca Raton,FL,33432.0,1815000.0,3.0,3.0,...,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Gregory Lynn,Lynn Realty Group,Ashley Israel,Coldwell Banker Realty /Delray Beach,Lisa McKibbin,Lokation,None,None
5,PAST SALE,2025-03-21,Condo/Co-op,3720 S Ocean Blvd #507,Highland Beach,FL,33487.0,1560000.0,2.0,2.5,...,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Jonathan Lewis,Elite Realty of South Florida,None,None,Laura Creech,Premier Brokers International,None,None
6,PAST SALE,2025-03-17,Condo/Co-op,801 S Olive Ave #1503,West Palm Beach,FL,33401.0,1550000.0,2.0,2.0,...,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Ginger Gowing,Luxury Real Estate Group LLC,None,None,Janet Gerds,Illustrated Properties,None,None
7,PAST SALE,2025-03-17,Condo/Co-op,131 Ocean Grande Blvd #402,Jupiter,FL,33477.0,1400000.0,3.0,3.5,...,-80.066425,https://www.redfin.com/FL/Jupiter/131-Ocean-Gr...,James Chivers,Premier Properties of South Fl,None,None,Kirstin Muller,Premier Properties of South Fl,None,None
8,PAST SALE,2025-03-18,Condo/Co-op,400 Seasage Dr #701,Delray Beach,FL,33483.0,1375000.0,2.0,2.0,...,-80.063637,https://www.redfin.com/FL/Delray-Beach/400-Sea...,Charles Lynn,One Sotheby's International Realty,None,None,Kim Lekas,"Premier Estate Properties, Inc",None,None
9,PAST SALE,2025-03-21,Condo/Co-op,120 S Ocean Blvd Unit 1-D,Delray Beach,FL,33483.0,1275000.0,2.0,2.0,...,-80.059459,https://www.redfin.com/FL/Delray-Beach/120-S-O...,Gary Shusas,Coldwell Banker Realty /Delray Beach,Nicholas Wisell,Coldwell Banker Realty /Delray Beach,Carol Corey,One Sotheby's International Realty,Patricia Burke,One Sotheby's International Realty


## Current Week's Values

In [22]:
# Number of condos sold
current_week_condo_sales = len(df2)
print(len(df2))

118


In [23]:
# Average sale price
average_sale_price = round(df2['PRICE'].mean())
print(f'${average_sale_price:,.0f}')

$527,811


In [24]:
# Average PSF
average_price_psf = round(df2['$/SQUARE FEET'].mean())
print(f'${average_price_psf:,.0f}')

$304


In [25]:
# Total sales dollar volume
current_week_condo_sales_total = df2['PRICE'].sum()
print(f'${current_week_condo_sales_total:,.0f}')

$62,281,725


In [26]:
def save_current_week_data_df(df, filename='last_week_data.csv'):
    df.to_csv(filename, index=False)

# Create a dictionary of your current week data.
current_week_data = {
    "Condo Sales": current_week_condo_sales,
    "Average Sale Price": average_sale_price,
    "Average PSF": average_price_psf,
    "Last Week Total Dollar Volume": current_week_condo_sales_total
}

# Convert the dictionary into a DataFrame.
current_week_summary_df = pd.DataFrame([current_week_data])

# Save the DataFrame to a CSV file.
save_current_week_data_df(current_week_summary_df)

# Next week, load last week's data automatically.
last_week_df = load_last_week_data_df()
if last_week_df is not None:
    print("Last week's data:")
    print(last_week_df)
else:
    print("No previous data found.")

Last week's data:
   Condo Sales  Average Sale Price  Average PSF  Last Week Total Dollar Volume
0          118              527811          304                     62281725.0


## Previous Week's Values

### The below 2 cells are only used if we need to back fill from an earlier week than just last week.

For example, if we start in a new locale and comparison information can't get pulled from an existing story

In [22]:
# # Define the date range
# start_date = '2024-12-23'
# end_date = '2024-12-29'

# # Filter the DataFrame between two dates (inclusive)
# old_week = df[(df['SOLD DATE'] >= start_date) & (df['SOLD DATE'] <= end_date)]

In [21]:
# # comparison week total sales
# print(f'Comparison week filtered sales: {(len(old_week))}')
# print('-----------')
# print(f'Comparison week average sales price: ${old_week["PRICE"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week average psf: ${old_week["$/SQUARE FEET"].mean():,.02f}')
# print('-----------')
# print(f'Comparison week sales total: ${old_week["PRICE"].sum():,.02f}')
# print('-----------')

In [46]:
print('Input Previous Week Condo Sales Total (number units sold):')
# previous_week_condo_sales = input()
previous_week_condo_sales = last_week['Condo Sales'].iloc[0]
print(previous_week_condo_sales)
print('-----------')

print('Input Previous Week Condo Average Sales Price:')
# previous_week_average_sales_price = input()
previous_week_average_sales_price = last_week['Average Sale Price'].iloc[0]
print(previous_week_average_sales_price)
print('-----------')

print('Input Previous Week Condo Average PSF:')
# previous_week_average_psf = input()
previous_week_average_psf = last_week['Average PSF'].iloc[0]
print(previous_week_average_psf)
print('-----------')

print('Input Previous Week Condo Sales total (ex: 198_000_000)')
# previous_week_condo_sales_total = input()
previous_week_condo_sales_total = last_week['Last Week Total Dollar Volume'].iloc[0]
print(previous_week_condo_sales_total)


## Clean Data

Input Previous Week Condo Sales Total (number units sold):
139
-----------
Input Previous Week Condo Average Sales Price:
693338
-----------
Input Previous Week Condo Average PSF:
387
-----------
Input Previous Week Condo Sales total (ex: 198_000_000)
96373968.0


In [36]:
# df_top_ten

## Correction section

In [10]:
# df.at[index#,'col_name']

## Format Data

In [47]:
### Insert NaNs if needed ###
df_top_ten = df_top_ten.replace('N/A', np.nan)

## Color-code top sale

In [48]:
### Insert RANK values ###
df_top_ten['RANK'] = range(1, len(df_top_ten) + 1)
# use numpy to assign values to the 'COLOR' column
df_top_ten['COLOR'] = np.where(df_top_ten['RANK'] <= 1, 'orange', 'blue')

## HTML Popup Formatter

In [49]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [50]:
pd.set_option('display.max_columns',None)

In [51]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-21,Condo/Co-op,1000 S Ocean Blvd #701,Boca Raton,FL,33432.0,13000000.0,3.0,3.5,One Thousand Ocean,6037.0,NaN,2010.0,NaN,2153.0,10357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Beaches MLS,RX-11045446,N,Y,26.336721,-80.071445,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Senada Adzem,Douglas Elliman,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,One Thousand Ocean at 1000 S Ocean Blvd #701 i...


In [52]:
def popup_html(row):
    address = row['ADDRESS']  
    price = row['PRICE']
    psf = row['$/SQUARE FEET']
    sf = row['SQUARE FEET']
    year_built = row['YEAR BUILT']
    
    html = '''<!DOCTYPE html>
    <html>
    <strong>Building Name: </strong>{}'''.format(address) + '''<br>
    <strong>Sale Price: </strong>${}'''.format(f'{price:,}') + '''<br>
    <strong>Price sq ft: </strong>${}'''.format(f'{psf:,}') + '''<br>
    <strong>Square Footage: </strong>{}'''.format(f'{sf:,}') + '''<br>
    <strong>Year Built: </strong>{}'''.format(f'{str(year_built)}') + '''<br>
    </html>
    '''
    return html

In [53]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-21,Condo/Co-op,1000 S Ocean Blvd #701,Boca Raton,FL,33432.0,13000000.0,3.0,3.5,One Thousand Ocean,6037.0,NaN,2010.0,NaN,2153.0,10357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Beaches MLS,RX-11045446,N,Y,26.336721,-80.071445,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Senada Adzem,Douglas Elliman,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,One Thousand Ocean at 1000 S Ocean Blvd #701 i...
1,PAST SALE,2025-03-20,Condo/Co-op,570 Ocean Dr #902,Juno Beach,FL,33408.0,3050000.0,3.0,4.5,Oceanfront AT Juno Beach Condo,3955.0,NaN,1995.0,NaN,771.0,4333.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Beaches MLS,RX-11020661,N,Y,26.874985,-80.052079,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Carla Christenson,One Sotheby's International Re,Richard Powers,One Sotheby's International Re,Andrea Roth,Echo Fine Properties,None,None,2,blue,Oceanfront AT Juno Beach Condo at 570 Ocean Dr...
2,PAST SALE,2025-03-20,Condo/Co-op,20155 Boca West Dr Unit C404,Boca Raton,FL,33434.0,2999000.0,2.0,2.5,Akoya Boca West,2003.0,NaN,2019.0,NaN,1497.0,3572.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11054302,N,Y,26.377420,-80.164122,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Nancy Ghen,Coldwell Banker/BR,None,None,Rebecca Spooner,Siemens Group Realty Corp.,None,None,3,blue,Akoya Boca West at 20155 Boca West Dr Unit C40...
3,PAST SALE,2025-03-21,Condo/Co-op,222 Lakeview Ave Unit 2s,West Palm Beach,FL,33401.0,2750000.0,3.0,3.5,Esperante Condo,2226.0,NaN,1989.0,NaN,1235.0,2677.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/222-...,Beaches MLS,RX-11052939,N,Y,26.705667,-80.050960,https://www.redfin.com/FL/West-Palm-Beach/222-...,Patrick Lafferty,Compass Florida LLC,None,None,Seth Koretzky,Compass Florida LLC,None,None,4,blue,Esperante Condo at 222 Lakeview Ave Unit 2s in...
4,PAST SALE,2025-03-17,Condo/Co-op,701 E Camino Real Unit 8-E,Boca Raton,FL,33432.0,1815000.0,3.0,3.0,Boca Inlet,1839.0,NaN,1965.0,NaN,987.0,2947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11042942,N,Y,26.339459,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Gregory Lynn,Lynn Realty Group,Ashley Israel,Coldwell Banker Realty /Delray Beach,Lisa McKibbin,Lokation,None,None,5,blue,Boca Inlet at 701 E Camino Real Unit 8-E in Bo...
5,PAST SALE,2025-03-21,Condo/Co-op,3720 S Ocean Blvd #507,Highland Beach,FL,33487.0,1560000.0,2.0,2.5,Toscana West Condo,2353.0,NaN,2001.0,NaN,663.0,2965.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11060009,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Jonathan Lewis,Elite Realty of South Florida,None,None,Laura Creech,Premier Brokers International,None,None,6,blue,Toscana West Condo at 3720 S Ocean Blvd #507 i...
6,PAST SALE,2025-03-17,Condo/Co-op,801 S Olive Ave #1503,West Palm Beach,FL,33401.0,1550000.0,2.0,2.0,One City Plaza Condo,1437.0,58651.0,2006.0,NaN,1079.0,1607.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-11036395,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Ginger Gowing,Luxury Real Estate Group LLC,None,None,Janet Gerds,Illustrated Properties,None,None,7,blue,One City Plaza Condo at 801 S Olive Ave #1503 ...
7,PAST SALE,2025-03-17,Condo/Co-op,131 Ocean Grande Blvd #402,Jupiter,FL,33477.0,1400000.0,3.0,3.5,Jupiter Ocean Grande 1 Condo,2146.0,NaN,2001.0,NaN,652.0,2067.0,Sold,NaN,NaN,htt

In [54]:
from folium.plugins import MarkerCluster

title_html = '''
              <h3 align="center" style="font-size:16px"><b>{}</b></h3>
             '''.format(f'Recent {county} County Condo Sales ')

caption_html = '''
                <p align="center" style="vertical-align: bottom; font-size:13px"><i>{}</i></p>
                '''.format(f'{week_starting} — {week_ending}')


### Create map container ###
m = folium.Map(location=df[["LATITUDE", "LONGITUDE"]].mean().to_list(),zoom_start=9.5,tiles=None)

# Create two FeatureGroups for different color pins
fg_blue = folium.FeatureGroup(name='All other sales')
fg_orange = folium.FeatureGroup(name='Top Sale')

for index, row in df_top_ten.iterrows():
    # Add the markers to the appropriate FeatureGroup based on the color
    if row['COLOR'] == 'blue':
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_blue)
    else:
        marker = folium.Marker(
            location=[row['LATITUDE'], row['LONGITUDE']],
            radius=5,
            fill=True,
            icon=folium.Icon(color=row['COLOR']),
            popup=folium.Popup(popup_html(row), max_width=400))
        marker.add_to(fg_orange)

# Add the FeatureGroups to the map
fg_orange.add_to(m)
fg_blue.add_to(m)

folium.TileLayer('OpenStreetMap',control=False).add_to(m)

# Add LayerControl to the map
folium.map.LayerControl(collapsed=False).add_to(m)
m.get_root().html.add_child(folium.Element(title_html))
m.get_root().html.add_child(folium.Element(caption_html))
            
# Display map
m

In [55]:
m.save('index.html')

## Data snagger

In [56]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

DAYS_MAX = '\033[1m' + 'Most Days on Market' + '\033[0m'
DAYS_MIN = '\033[1m' + 'Fewest Days on Market' + '\033[0m'

In [57]:
df_top_ten.head(1)

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-21,Condo/Co-op,1000 S Ocean Blvd #701,Boca Raton,FL,33432.0,13000000.0,3.0,3.5,One Thousand Ocean,6037.0,NaN,2010.0,NaN,2153.0,10357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Beaches MLS,RX-11045446,N,Y,26.336721,-80.071445,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Senada Adzem,Douglas Elliman,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,One Thousand Ocean at 1000 S Ocean Blvd #701 i...


In [122]:
# # Highest and lowest sale price
# highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
# lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

# print(f"Highest Sale Price: {df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_sale_price_idx]['DAYS ON MARKET']}")
# print(f"Lowest Sale Price: {df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_sale_price_idx]['DAYS ON MARKET']}")

# # Highest and lowest psf
# highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
# lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

# print(f"Highest PSF: {df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[highest_psf_idx]['DAYS ON MARKET']}")
# print(f"Lowest PSF: {df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Days on market: {df_top_ten.loc[lowest_psf_idx]['DAYS ON MARKET']}")

## Map URL snagger

Map template URL: `https://trd-digital.github.io/trd-news-interactive-maps/{map-folder-name}`

In [58]:
base_name = 'https://trd-digital.github.io/trd-news-interactive-maps/'

cwd = os.getcwd()

cwd = cwd.split('/')

final_name = base_name + cwd[-1]
print(final_name)

https://trd-digital.github.io/trd-news-interactive-maps/PalmBeach_County_condo_sales_week_ending_03242024


# CREATE TEMPLATE 

In [59]:
muni_set = set(df_top_ten['CITY'])

In [60]:
df_top_ten['FULL_NAME'] = df_top_ten['LOCATION'] + ' at ' + df_top_ten['ADDRESS'] + ' in ' + df_top_ten['CITY']

In [61]:
df_top_ten.reset_index(inplace=True,drop=True)

In [62]:
top_sale = df_top_ten.at[0,'FULL_NAME']
second_top_sale = df_top_ten.at[1,'FULL_NAME']

In [63]:
top_sale

'One Thousand Ocean at 1000 S Ocean Blvd #701 in Boca Raton'

In [64]:
### Average Sales Price ###
if float(average_sale_price) > float(previous_week_average_sales_price):
    change = 'higher'
    difference = float(average_sale_price) - float(previous_week_average_sales_price)
else:
    change = 'lower'
    difference = float(previous_week_average_sales_price) - float(average_sale_price)
    
### Total condo dollar volume ###
if float(current_week_condo_sales_total) > float(previous_week_condo_sales_total):
    seo_head = 'Rises'
elif float(previous_week_condo_sales_total) < float(current_week_condo_sales_total):
    seo_head = 'Falls'
else:
    seo_head = 'Unchanged'
    
### Average PSF ###    
if float(average_price_psf) > float(previous_week_average_psf):
    psf_rf = 'rose'
elif float(average_price_psf) < float(previous_week_average_psf):
    psf_rf ='fell'
else:
    psf_rf = 'equaled'

In [65]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [66]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-21,Condo/Co-op,1000 S Ocean Blvd #701,Boca Raton,FL,33432.0,13000000.0,3.0,3.5,One Thousand Ocean,6037.0,NaN,2010.0,NaN,2153.0,10357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Beaches MLS,RX-11045446,N,Y,26.336721,-80.071445,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Senada Adzem,Douglas Elliman,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,One Thousand Ocean at 1000 S Ocean Blvd #701 i...
1,PAST SALE,2025-03-20,Condo/Co-op,570 Ocean Dr #902,Juno Beach,FL,33408.0,3050000.0,3.0,4.5,Oceanfront AT Juno Beach Condo,3955.0,NaN,1995.0,NaN,771.0,4333.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Beaches MLS,RX-11020661,N,Y,26.874985,-80.052079,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Carla Christenson,One Sotheby's International Re,Richard Powers,One Sotheby's International Re,Andrea Roth,Echo Fine Properties,None,None,2,blue,Oceanfront AT Juno Beach Condo at 570 Ocean Dr...
2,PAST SALE,2025-03-20,Condo/Co-op,20155 Boca West Dr Unit C404,Boca Raton,FL,33434.0,2999000.0,2.0,2.5,Akoya Boca West,2003.0,NaN,2019.0,NaN,1497.0,3572.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11054302,N,Y,26.377420,-80.164122,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Nancy Ghen,Coldwell Banker/BR,None,None,Rebecca Spooner,Siemens Group Realty Corp.,None,None,3,blue,Akoya Boca West at 20155 Boca West Dr Unit C40...
3,PAST SALE,2025-03-21,Condo/Co-op,222 Lakeview Ave Unit 2s,West Palm Beach,FL,33401.0,2750000.0,3.0,3.5,Esperante Condo,2226.0,NaN,1989.0,NaN,1235.0,2677.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/222-...,Beaches MLS,RX-11052939,N,Y,26.705667,-80.050960,https://www.redfin.com/FL/West-Palm-Beach/222-...,Patrick Lafferty,Compass Florida LLC,None,None,Seth Koretzky,Compass Florida LLC,None,None,4,blue,Esperante Condo at 222 Lakeview Ave Unit 2s in...
4,PAST SALE,2025-03-17,Condo/Co-op,701 E Camino Real Unit 8-E,Boca Raton,FL,33432.0,1815000.0,3.0,3.0,Boca Inlet,1839.0,NaN,1965.0,NaN,987.0,2947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11042942,N,Y,26.339459,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Gregory Lynn,Lynn Realty Group,Ashley Israel,Coldwell Banker Realty /Delray Beach,Lisa McKibbin,Lokation,None,None,5,blue,Boca Inlet at 701 E Camino Real Unit 8-E in Bo...
5,PAST SALE,2025-03-21,Condo/Co-op,3720 S Ocean Blvd #507,Highland Beach,FL,33487.0,1560000.0,2.0,2.5,Toscana West Condo,2353.0,NaN,2001.0,NaN,663.0,2965.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11060009,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Jonathan Lewis,Elite Realty of South Florida,None,None,Laura Creech,Premier Brokers International,None,None,6,blue,Toscana West Condo at 3720 S Ocean Blvd #507 i...
6,PAST SALE,2025-03-17,Condo/Co-op,801 S Olive Ave #1503,West Palm Beach,FL,33401.0,1550000.0,2.0,2.0,One City Plaza Condo,1437.0,58651.0,2006.0,NaN,1079.0,1607.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-11036395,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Ginger Gowing,Luxury Real Estate Group LLC,None,None,Janet Gerds,Illustrated Properties,None,None,7,blue,One City Plaza Condo at 801 S Olive Ave #1503 ...
7,PAST SALE,2025-03-17,Condo/Co-op,131 Ocean Grande Blvd #402,Jupiter,FL,33477.0,1400000.0,3.0,3.5,Jupiter Ocean Grande 1 Condo,2146.0,NaN,2001.0,NaN,652.0,2067.0,Sold,NaN,NaN,htt

In [67]:
story_string = f'''
\033[1mHED:\033[0m {df_top_ten.loc[df_top_ten['PRICE'].idxmax()]['LOCATION']} closing tops {county} weekly condo sales 
\033[1mDEK:\033[0m Top 10 sales ranged from ${df_top_ten.at[9,'PRICE']:,.0f} to ${df_top_ten.at[0,'PRICE']:,.0f}
\033[1mFEATURED HED:\033[0m
\033[1mSEO HED:\033[0m {county} County Weekly Condo Report 
\033[1mSEO DESCRIPTION:\033[0m {county} County’s condo sales XXXXXXX
\033[1mAUTHOR:\033[0m Kate Hinsche
\033[1mRESEARCH:\033[0m Adam Farence
\033[1mSocial:\033[0m #{county} condo sales XXXXX
\033[1mART:\033[0m

Image of the top sale

*Please provide credits for any images that you share
\033[1mSTORY TYPE:\033[0m Report
\033[1mSECTOR\033[0m (formerly CATEGORY): Residential Real Estate
\033[1mTAGS:\033[0m condo sales, {county}, weekly condo sales, {', '.join(str(x) for x in muni_set)}, condos, condo market

\033[1mNeighborhood:\033[0m 
\033[1mProperty:\033[0m
\033[1mProperty Type:\033[0m
\033[1mCompanies:\033[0m 
\033[1mPeople:\033[0m
\033[1mIssues:\033[0m
\033[1mRegion:\033[0m
\033[1m[]Show in Yahoo Feed\033[0m

{county} County’s NEWS PEG HERE.

Brokers closed {int(current_week_condo_sales):,.0f} condo sales totaling ${float(current_week_condo_sales_total):,.0f} million from {week_starting} to {week_ending}. The previous week, brokers closed {previous_week_condo_sales} condo sales totaling ${int(previous_week_condo_sales_total):,.0f}.

Last week’s units sold for an average of ${float(average_sale_price):,.0f}, {change} than the ${float(previous_week_average_sales_price):,.0f} average price from the previous week. The average price per square foot {psf_rf} to ${average_price_psf} from ${previous_week_average_psf}, according to data from Redfin.

For the top 10 sales, prices ranged from ${df_top_ten.at[9,'PRICE']:,} million to ${df_top_ten.at[0,'PRICE']:,} million. 

{top_sale} took the top spot with a ${df_top_ten.at[0,'PRICE']:,} million closing. The sale closed at ${df_top_ten.at[0,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[0,'list_agent_name_1']} with {df_top_ten.at[0,'list_broker_name_1']}, and {df_top_ten.at[0,'list_agent_name_2']} with {df_top_ten.at[0,'list_broker_name_2']} had the listing.
{df_top_ten.at[0,'buy_agent_name_1']} with {df_top_ten.at[0,'buy_broker_name_1']}, and {df_top_ten.at[0,'buy_agent_name_2']} with {df_top_ten.at[0,'buy_broker_name_2']} represented the buyer.

{second_top_sale}, closed for the second highest amount, ${df_top_ten.at[1,'PRICE']:,}, or ${df_top_ten.at[1,'$/SQUARE FEET']:,} per square foot.

{df_top_ten.at[1,'list_agent_name_1']} with {df_top_ten.at[1,'list_broker_name_1']}, and {df_top_ten.at[1,'list_agent_name_2']} with {df_top_ten.at[1,'list_broker_name_2']} had the listing.
{df_top_ten.at[1,'buy_agent_name_1']} with {df_top_ten.at[1,'buy_broker_name_1']}, and {df_top_ten.at[1,'buy_agent_name_2']} with {df_top_ten.at[1,'buy_broker_name_2']} represented the buyer.

<figure>
 <div class="container">
   <div class="iframe-wrap">
   <iframe src="{final_name}" width="100%" height="600" frameBorder="0" scrolling="no"></iframe>
  </div>
</div>
  <figcaption align="right"><a href="https://leafletjs.com/">Leaflet</a> map created by Adam Farence | Data by © <a href="https://www.openstreetmap.org/#map=4/38.01/-95.84"> OpenStreetMap</a>, under <a href="https://www.openstreetmap.org/copyright">ODbl.</a></figcaption>
</figure>

Here’s a breakdown of the top 10 sales from {week_starting} to {week_ending}:
'''

print(story_string)


HED: One Thousand Ocean closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,275,000 to $13,000,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Jupiter, Highland Beach, West Palm Beach, Boca Raton, Delray Beach, Juno Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 118 condo sales totaling $62,281,725 million from Mar. 16th to Mar. 23rd. The previous week, brokers closed 139 condo sales totaling $96,373,968.

Last week’s units sold for an

In [68]:
story_checklist = '''
\033[1mRemember to...:\033[0m

1. Double check all names. Sometimes names differ between the hed and the body of the story.
    For example, "Surf Club Four Seasons" in the HED and "Four Seasons Residences at the Surfclub"
    in the body.
    
2. Add in context, if available. When there's a high-priced condo sale, check and see if there's
    a story. If there is, add in some extra details and link back to it.
'''

In [69]:
### Set up formatting ###
BR = '\n'

ME = '\033[1m' + 'Most Expensive' + '\033[0m'
LE = '\033[1m' + 'Least Expensive' + '\033[0m'

MAX_PSF = '\033[1m' + 'Highest Price Per Square Foot' + '\033[0m'
MIN_PSF = '\033[1m' + 'Lowest Price Per Square Foot' + '\033[0m'

In [70]:
df_top_ten.columns

Index(['SALE TYPE', 'SOLD DATE', 'PROPERTY TYPE', 'ADDRESS', 'CITY',
       'STATE OR PROVINCE', 'ZIP OR POSTAL CODE', 'PRICE', 'BEDS', 'BATHS',
       'LOCATION', 'SQUARE FEET', 'LOT SIZE', 'YEAR BUILT', 'DAYS ON MARKET',
       '$/SQUARE FEET', 'HOA/MONTH', 'STATUS', 'NEXT OPEN HOUSE START TIME',
       'NEXT OPEN HOUSE END TIME', 'URL', 'SOURCE', 'MLS#', 'FAVORITE',
       'INTERESTED', 'LATITUDE', 'LONGITUDE', 'url', 'list_agent_name_1',
       'list_broker_name_1', 'list_agent_name_2', 'list_broker_name_2',
       'buy_agent_name_1', 'buy_broker_name_1', 'buy_agent_name_2',
       'buy_broker_name_2', 'RANK', 'COLOR', 'FULL_NAME'],
      dtype='object')

In [71]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-21,Condo/Co-op,1000 S Ocean Blvd #701,Boca Raton,FL,33432.0,13000000.0,3.0,3.5,One Thousand Ocean,6037.0,NaN,2010.0,NaN,2153.0,10357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Beaches MLS,RX-11045446,N,Y,26.336721,-80.071445,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Senada Adzem,Douglas Elliman,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,One Thousand Ocean at 1000 S Ocean Blvd #701 i...
1,PAST SALE,2025-03-20,Condo/Co-op,570 Ocean Dr #902,Juno Beach,FL,33408.0,3050000.0,3.0,4.5,Oceanfront AT Juno Beach Condo,3955.0,NaN,1995.0,NaN,771.0,4333.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Beaches MLS,RX-11020661,N,Y,26.874985,-80.052079,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Carla Christenson,One Sotheby's International Re,Richard Powers,One Sotheby's International Re,Andrea Roth,Echo Fine Properties,None,None,2,blue,Oceanfront AT Juno Beach Condo at 570 Ocean Dr...
2,PAST SALE,2025-03-20,Condo/Co-op,20155 Boca West Dr Unit C404,Boca Raton,FL,33434.0,2999000.0,2.0,2.5,Akoya Boca West,2003.0,NaN,2019.0,NaN,1497.0,3572.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11054302,N,Y,26.377420,-80.164122,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Nancy Ghen,Coldwell Banker/BR,None,None,Rebecca Spooner,Siemens Group Realty Corp.,None,None,3,blue,Akoya Boca West at 20155 Boca West Dr Unit C40...
3,PAST SALE,2025-03-21,Condo/Co-op,222 Lakeview Ave Unit 2s,West Palm Beach,FL,33401.0,2750000.0,3.0,3.5,Esperante Condo,2226.0,NaN,1989.0,NaN,1235.0,2677.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/222-...,Beaches MLS,RX-11052939,N,Y,26.705667,-80.050960,https://www.redfin.com/FL/West-Palm-Beach/222-...,Patrick Lafferty,Compass Florida LLC,None,None,Seth Koretzky,Compass Florida LLC,None,None,4,blue,Esperante Condo at 222 Lakeview Ave Unit 2s in...
4,PAST SALE,2025-03-17,Condo/Co-op,701 E Camino Real Unit 8-E,Boca Raton,FL,33432.0,1815000.0,3.0,3.0,Boca Inlet,1839.0,NaN,1965.0,NaN,987.0,2947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11042942,N,Y,26.339459,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Gregory Lynn,Lynn Realty Group,Ashley Israel,Coldwell Banker Realty /Delray Beach,Lisa McKibbin,Lokation,None,None,5,blue,Boca Inlet at 701 E Camino Real Unit 8-E in Bo...
5,PAST SALE,2025-03-21,Condo/Co-op,3720 S Ocean Blvd #507,Highland Beach,FL,33487.0,1560000.0,2.0,2.5,Toscana West Condo,2353.0,NaN,2001.0,NaN,663.0,2965.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11060009,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Jonathan Lewis,Elite Realty of South Florida,None,None,Laura Creech,Premier Brokers International,None,None,6,blue,Toscana West Condo at 3720 S Ocean Blvd #507 i...
6,PAST SALE,2025-03-17,Condo/Co-op,801 S Olive Ave #1503,West Palm Beach,FL,33401.0,1550000.0,2.0,2.0,One City Plaza Condo,1437.0,58651.0,2006.0,NaN,1079.0,1607.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-11036395,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Ginger Gowing,Luxury Real Estate Group LLC,None,None,Janet Gerds,Illustrated Properties,None,None,7,blue,One City Plaza Condo at 801 S Olive Ave #1503 ...
7,PAST SALE,2025-03-17,Condo/Co-op,131 Ocean Grande Blvd #402,Jupiter,FL,33477.0,1400000.0,3.0,3.5,Jupiter Ocean Grande 1 Condo,2146.0,NaN,2001.0,NaN,652.0,2067.0,Sold,NaN,NaN,htt

In [72]:
print(story_string)

# Highest and lowest sale price
highest_sale_price_idx = df_top_ten['PRICE'].idxmax()
lowest_sale_price_idx = df_top_ten['PRICE'].idxmin()

print(f"{ME}{BR}{df_top_ten.loc[highest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[highest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[highest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[highest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_sale_price_idx]['buy_broker_name_2']}\n")
print(f"{LE}{BR}{df_top_ten.loc[lowest_sale_price_idx]['LOCATION']}, {df_top_ten.loc[lowest_sale_price_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_sale_price_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_sale_price_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_sale_price_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_sale_price_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_sale_price_idx]['buy_broker_name_2']}\n")

# Highest and lowest psf
highest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmax()
lowest_psf_idx = df_top_ten['$/SQUARE FEET'].idxmin()

print(f"{MAX_PSF}{BR}{df_top_ten.loc[highest_psf_idx]['LOCATION']}, {df_top_ten.loc[highest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[highest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[highest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[highest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[highest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[highest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[highest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[highest_psf_idx]['buy_broker_name_2']}\n")
print(f"{MIN_PSF}{BR}{df_top_ten.loc[lowest_psf_idx]['LOCATION']}, {df_top_ten.loc[lowest_psf_idx]['ADDRESS']}, in {df_top_ten.loc[lowest_psf_idx]['CITY']} | Price: ${df_top_ten.loc[lowest_psf_idx]['PRICE']:,.0f} | ${df_top_ten.loc[lowest_psf_idx]['$/SQUARE FEET']:,.0f} psf | Listing agents: {df_top_ten.loc[lowest_psf_idx]['list_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['list_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['list_broker_name_2']} | Buyer's agents: {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_1']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_1']} and {df_top_ten.loc[lowest_psf_idx]['buy_agent_name_2']} with {df_top_ten.loc[lowest_psf_idx]['buy_broker_name_2']}\n")


print("\n--------SALE LINKS FOR REFERENCE ONLY--------\n")
print('Top sale: ',df_top_ten['URL'].iloc[0])
print('Second priciest sale: ', df_top_ten['URL'].iloc[1])
print('Cheapest top ten sale: ',df_top_ten['URL'].iloc[-1])
print('Second Cheapest top ten sale: ',df_top_ten['URL'].iloc[-2])
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print('Highest top ten psf: ',max_url)

# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print('Lowest top ten psf: ', min_url)


HED: One Thousand Ocean closing tops Palm Beach County weekly condo sales 
DEK: Top 10 sales ranged from $1,275,000 to $13,000,000
FEATURED HED:
SEO HED: Palm Beach County County Weekly Condo Report 
SEO DESCRIPTION: Palm Beach County County’s condo sales XXXXXXX
AUTHOR: Kate Hinsche
RESEARCH: Adam Farence
Social: #Palm Beach County condo sales XXXXX
ART:

Image of the top sale

*Please provide credits for any images that you share
STORY TYPE: Report
SECTOR (formerly CATEGORY): Residential Real Estate
TAGS: condo sales, Palm Beach County, weekly condo sales, Jupiter, Highland Beach, West Palm Beach, Boca Raton, Delray Beach, Juno Beach, condos, condo market

Neighborhood: 
Property:
Property Type:
Companies: 
People:
Issues:
Region:
[]Show in Yahoo Feed

Palm Beach County County’s NEWS PEG HERE.

Brokers closed 118 condo sales totaling $62,281,725 million from Mar. 16th to Mar. 23rd. The previous week, brokers closed 139 condo sales totaling $96,373,968.

Last week’s units sold for an

In [80]:
print('Top sale:', df_top_ten['URL'].iloc[0])

Top sale: https://www.redfin.com/FL/Boca-Raton/1000-S-Ocean-Blvd-33432/unit-701/home/42663639


In [74]:
print(df_top_ten['URL'].iloc[1])

https://www.redfin.com/FL/Juno-Beach/570-Ocean-Dr-33408/unit-902/home/42396491


In [75]:
print(df_top_ten['URL'].iloc[-1])

https://www.redfin.com/FL/Delray-Beach/120-S-Ocean-Blvd-33483/unit-1D/home/42365587


In [76]:
print(df_top_ten['URL'].iloc[-2])

https://www.redfin.com/FL/Delray-Beach/400-Seasage-Dr-33483/unit-701/home/42368929


In [77]:
# Find the index of the row with the maximum '$/SQUARE FEET' value
max_index = df_top_ten['$/SQUARE FEET'].idxmax()

# Use the index to access the 'URL' of that row
max_url = df_top_ten.loc[max_index, 'URL']

print(max_url)

https://www.redfin.com/FL/Boca-Raton/1000-S-Ocean-Blvd-33432/unit-701/home/42663639


In [78]:
# Find the index of the row with the minimum '$/SQUARE FEET' value
min_index = df_top_ten['$/SQUARE FEET'].idxmin()

# Use the index to access the 'URL' of that row
min_url = df_top_ten.loc[min_index, 'URL']

print(min_url)

https://www.redfin.com/FL/Jupiter/131-Ocean-Grande-Blvd-33477/unit-402/home/42555860


In [79]:
df_top_ten

,SALE TYPE,SOLD DATE,PROPERTY TYPE,ADDRESS,CITY,STATE OR PROVINCE,ZIP OR POSTAL CODE,PRICE,BEDS,BATHS,LOCATION,SQUARE FEET,LOT SIZE,YEAR BUILT,DAYS ON MARKET,$/SQUARE FEET,HOA/MONTH,STATUS,NEXT OPEN HOUSE START TIME,NEXT OPEN HOUSE END TIME,URL,SOURCE,MLS#,FAVORITE,INTERESTED,LATITUDE,LONGITUDE,url,list_agent_name_1,list_broker_name_1,list_agent_name_2,list_broker_name_2,buy_agent_name_1,buy_broker_name_1,buy_agent_name_2,buy_broker_name_2,RANK,COLOR,FULL_NAME
0,PAST SALE,2025-03-21,Condo/Co-op,1000 S Ocean Blvd #701,Boca Raton,FL,33432.0,13000000.0,3.0,3.5,One Thousand Ocean,6037.0,NaN,2010.0,NaN,2153.0,10357.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Beaches MLS,RX-11045446,N,Y,26.336721,-80.071445,https://www.redfin.com/FL/Boca-Raton/1000-S-Oc...,Senada Adzem,Douglas Elliman,None,None,Jonathan Postma,Coldwell Banker/BR,None,None,1,orange,One Thousand Ocean at 1000 S Ocean Blvd #701 i...
1,PAST SALE,2025-03-20,Condo/Co-op,570 Ocean Dr #902,Juno Beach,FL,33408.0,3050000.0,3.0,4.5,Oceanfront AT Juno Beach Condo,3955.0,NaN,1995.0,NaN,771.0,4333.0,Sold,NaN,NaN,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Beaches MLS,RX-11020661,N,Y,26.874985,-80.052079,https://www.redfin.com/FL/Juno-Beach/570-Ocean...,Carla Christenson,One Sotheby's International Re,Richard Powers,One Sotheby's International Re,Andrea Roth,Echo Fine Properties,None,None,2,blue,Oceanfront AT Juno Beach Condo at 570 Ocean Dr...
2,PAST SALE,2025-03-20,Condo/Co-op,20155 Boca West Dr Unit C404,Boca Raton,FL,33434.0,2999000.0,2.0,2.5,Akoya Boca West,2003.0,NaN,2019.0,NaN,1497.0,3572.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Beaches MLS,RX-11054302,N,Y,26.377420,-80.164122,https://www.redfin.com/FL/Boca-Raton/20155-Boc...,Nancy Ghen,Coldwell Banker/BR,None,None,Rebecca Spooner,Siemens Group Realty Corp.,None,None,3,blue,Akoya Boca West at 20155 Boca West Dr Unit C40...
3,PAST SALE,2025-03-21,Condo/Co-op,222 Lakeview Ave Unit 2s,West Palm Beach,FL,33401.0,2750000.0,3.0,3.5,Esperante Condo,2226.0,NaN,1989.0,NaN,1235.0,2677.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/222-...,Beaches MLS,RX-11052939,N,Y,26.705667,-80.050960,https://www.redfin.com/FL/West-Palm-Beach/222-...,Patrick Lafferty,Compass Florida LLC,None,None,Seth Koretzky,Compass Florida LLC,None,None,4,blue,Esperante Condo at 222 Lakeview Ave Unit 2s in...
4,PAST SALE,2025-03-17,Condo/Co-op,701 E Camino Real Unit 8-E,Boca Raton,FL,33432.0,1815000.0,3.0,3.0,Boca Inlet,1839.0,NaN,1965.0,NaN,987.0,2947.0,Sold,NaN,NaN,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Beaches MLS,RX-11042942,N,Y,26.339459,-80.076301,https://www.redfin.com/FL/Boca-Raton/701-E-Cam...,Gregory Lynn,Lynn Realty Group,Ashley Israel,Coldwell Banker Realty /Delray Beach,Lisa McKibbin,Lokation,None,None,5,blue,Boca Inlet at 701 E Camino Real Unit 8-E in Bo...
5,PAST SALE,2025-03-21,Condo/Co-op,3720 S Ocean Blvd #507,Highland Beach,FL,33487.0,1560000.0,2.0,2.5,Toscana West Condo,2353.0,NaN,2001.0,NaN,663.0,2965.0,Sold,NaN,NaN,https://www.redfin.com/FL/Highland-Beach/3720-...,Beaches MLS,RX-11060009,N,Y,26.408254,-80.066146,https://www.redfin.com/FL/Highland-Beach/3720-...,Jonathan Lewis,Elite Realty of South Florida,None,None,Laura Creech,Premier Brokers International,None,None,6,blue,Toscana West Condo at 3720 S Ocean Blvd #507 i...
6,PAST SALE,2025-03-17,Condo/Co-op,801 S Olive Ave #1503,West Palm Beach,FL,33401.0,1550000.0,2.0,2.0,One City Plaza Condo,1437.0,58651.0,2006.0,NaN,1079.0,1607.0,Sold,NaN,NaN,https://www.redfin.com/FL/West-Palm-Beach/801-...,Beaches MLS,RX-11036395,N,Y,26.705820,-80.052488,https://www.redfin.com/FL/West-Palm-Beach/801-...,Ginger Gowing,Luxury Real Estate Group LLC,None,None,Janet Gerds,Illustrated Properties,None,None,7,blue,One City Plaza Condo at 801 S Olive Ave #1503 ...
7,PAST SALE,2025-03-17,Condo/Co-op,131 Ocean Grande Blvd #402,Jupiter,FL,33477.0,1400000.0,3.0,3.5,Jupiter Ocean Grande 1 Condo,2146.0,NaN,2001.0,NaN,652.0,2067.0,Sold,NaN,NaN,htt

## Time on Market Calculator

In [94]:
################ YEAR, MONTH, DAY #######################

date1 = datetime(2024, 9, 19) ## List (Earlier) date
date2 = datetime(2025, 3, 20) ## Close (Later) date

delta = date2 - date1
num_days = delta.days

print(num_days)

182


## Clean CSV for Datawrapper Chart

In [82]:
chart_df = df_top_ten

In [83]:
chart_df = chart_df.fillna(" ")

In [85]:
chart_df['LISTING INFO'] = chart_df['list_agent_name_1'] + ' ' + chart_df['list_broker_name_1'] + ' ' + chart_df['list_agent_name_2'] + ' ' + chart_df['list_broker_name_2'] 
chart_df['BUYING INFO'] = chart_df['buy_agent_name_1'] + ' ' + chart_df['buy_broker_name_1'] + ' ' + chart_df['buy_agent_name_2'] + ' ' + chart_df['buy_broker_name_2'] 


In [86]:
chart_df['ADDRESS'] = chart_df['ADDRESS'] + ' in ' + chart_df['CITY']

In [87]:
columns_to_drop = [
    "SALE TYPE",
    "PROPERTY TYPE",
    "STATE OR PROVINCE",
    "ZIP OR POSTAL CODE",
    "LOT SIZE",
    "DAYS ON MARKET",
    "HOA/MONTH",
    "STATUS",
    "NEXT OPEN HOUSE START TIME",
    "NEXT OPEN HOUSE END TIME",
    "URL",
    "FAVORITE",
    "INTERESTED",
    "LATITUDE",
    "LONGITUDE",
    "url",
    "RANK",
    "COLOR",
    "FULL_NAME",
    "list_agent_name_1",
    "list_broker_name_1",
    "list_agent_name_2",
    "list_broker_name_2",
    "buy_agent_name_1",
    "buy_broker_name_1",
    "buy_agent_name_2",
    "buy_broker_name_2",
    "CITY",
    "LOCATION"
]

In [88]:
chart_df = chart_df.drop(columns=columns_to_drop)

In [89]:
chart_df['$/SQUARE FEET'].astype(float)

0    2153.0
1     771.0
2    1497.0
3    1235.0
4     987.0
5     663.0
6    1079.0
7     652.0
8     781.0
9     727.0
Name: $/SQUARE FEET, dtype: float64

In [90]:
chart_df['PRICE'] = chart_df['PRICE'].apply(lambda x: "${:,.0f}".format(x))
chart_df['$/SQUARE FEET'] = chart_df['$/SQUARE FEET'].apply(lambda x: "${:,.0f}".format(x) if pd.notnull(x) else x)
chart_df['SQUARE FEET'] = chart_df['SQUARE FEET'].apply(lambda x: "{:,.0f}".format(x))
chart_df['YEAR BUILT'] = chart_df['YEAR BUILT'].apply(lambda x: "{:.0f}".format(x))
chart_df['BEDS'] = chart_df['BEDS'].apply(lambda x: "{:.0f}".format(x))

In [91]:
csv_date_string = today.strftime("%m_%d_%Y")

In [92]:
chart_df.to_csv(f"WCS_Chart_{csv_date_string}.csv",index=False)

In [93]:
chart_df

,SOLD DATE,ADDRESS,PRICE,BEDS,BATHS,SQUARE FEET,YEAR BUILT,$/SQUARE FEET,SOURCE,MLS#,LISTING INFO,BUYING INFO
0,2025-03-21,1000 S Ocean Blvd #701 in Boca Raton,"$13,000,000",3,3.5,"6,037",2010,"$2,153",Beaches MLS,RX-11045446,Senada Adzem Douglas Elliman,Jonathan Postma Coldwell Banker/BR
1,2025-03-20,570 Ocean Dr #902 in Juno Beach,"$3,050,000",3,4.5,"3,955",1995,$771,Beaches MLS,RX-11020661,Carla Christenson One Sotheby's International ...,Andrea Roth Echo Fine Properties
2,2025-03-20,20155 Boca West Dr Unit C404 in Boca Raton,"$2,999,000",2,2.5,"2,003",2019,"$1,497",Beaches MLS,RX-11054302,Nancy Ghen Coldwell Banker/BR,Rebecca Spooner Siemens Group Realty Corp.
3,2025-03-21,222 Lakeview Ave Unit 2s in West Palm Beach,"$2,750,000",3,3.5,"2,226",1989,"$1,235",Beaches MLS,RX-11052939,Patrick Lafferty Compass Florida LLC,Seth Koretzky Compass Florida LLC
4,2025-03-17,701 E Camino Real Unit 8-E in Boca Raton,"$1,815,000",3,3.0,"1,839",1965,$987,Beaches MLS,RX-11042942,Gregory Lynn Lynn Realty Group Ashley Israel C...,Lisa McKibbin Lokation
5,2025-03-21,3720 S Ocean Blvd #507 in Highland Beach,"$1,560,000",2,2.5,"2,353",2001,$663,Beaches MLS,RX-11060009,Jonathan Lewis Elite Realty of South Florida,Laura Creech Premier Brokers International
6,2025-03-17,801 S Olive Ave #1503 in West Palm Beach,"$1,550,000",2,2.0,"1,437",2006,"$1,079",Beaches MLS,RX-11036395,Ginger Gowing Luxury Real Estate Group LLC,Janet Gerds Illustrated Properties
7,2025-03-17,131 Ocean Grande Blvd #402 in Jupiter,"$1,400,000",3,3.5,"2,146",2001,$652,Beaches MLS,RX-11016851,James Chivers Premier Properties of South Fl,Kirstin Muller Premier Properties of South Fl
8,2025-03-18,400 Seasage Dr #701 in Delray Beach,"$1,375,000",2,2.0,"1,760",1969,$781,Beaches MLS,RX-11040968,Charles Lynn One Sotheby's International Realt...,"Kim Lekas Premier Estate Properties, Inc"
9,2025-03-21,120 S Ocean Blvd Unit 1-D in Delray Beach,"$1,275,000",2,2.0,"1,753",1981,$727,Beaches MLS,RX-10988688,Gary Shusas Coldwell Banker Realty /Delray Bea...,Carol Corey One Sotheby's International Realty...
